In [1]:
# !pip install psycopg2
# !pip install google-cloud-sql

In [2]:
import pandas as pd
import numpy as np
import pickle
import json
import re

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import psycopg2

In [3]:
# #@markdown Please fill in the value below with your GCP project ID and then run the cell.

# # Please fill in these values.
# project_id = "digital-display-376201" #@param {type:"string"}

# # Quick input validations.
# assert project_id, "⚠️ Please provide a Google Cloud project ID"

# # Configure gcloud.
# !gcloud config set project {project_id}

In [4]:
# from google.cloud.sql.connector import Connector
# import sqlalchemy

In [5]:
# preparamos o driver do Chrome
driver = webdriver.Chrome('path/chromedriver')

<ipython-input-5-c16ba3ec765d>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('path/chromedriver')


In [6]:
!dir

 Volume in drive C is OS
 Volume Serial Number is 2CDA-BFF4

 Directory of C:\Users\Ernesto\OneDrive - usp.br\Turing USP\DS\data_engineering\crawler

02/28/2023  08:50 PM    <DIR>          .
02/28/2023  08:50 PM    <DIR>          ..
12/11/2022  11:15 AM    <DIR>          .ipynb_checkpoints
10/19/2022  06:46 PM        12,188,160 chromedriver.exe
11/22/2022  04:22 PM         6,776,586 chromedriver_win32.zip
01/31/2023  12:23 AM    <DIR>          data_input
02/28/2023  12:57 AM    <DIR>          data_output
11/22/2022  07:06 PM           560,484 foo.png
02/28/2023  12:17 AM         4,523,105 parte00.csv
02/28/2023  12:17 AM         4,608,220 parte01.csv
02/28/2023  12:17 AM         4,644,078 parte02.csv
02/28/2023  12:17 AM         4,656,313 parte03.csv
02/28/2023  12:17 AM         4,667,530 parte04.csv
02/28/2023  12:17 AM         4,695,746 parte05.csv
02/28/2023  12:17 AM         4,668,593 parte06.csv
02/28/2023  12:17 AM         4,660,944 parte07.csv
02/28/2023  12:17 AM         4,651,

In [7]:
file = 'parte00.csv'
# file = 'data_output/ztrending_movies.csv'

df0 = pd.read_csv(file, header=None).head(30)

In [8]:
df0 = df0.join(df0[1].apply(json.loads).apply(pd.Series))

In [9]:
df0 = df0.loc[:,['id','original_title']]

In [10]:
# df0.id

# Criacao do dataset com os dados do Rotten Tomatoes

In [11]:
# criamos esta funcao para que os nomes dos filmes sejam transformados nos nomes dos filmes da url
# exemplo: The Matrix ---> the_matrix, desta forma conseguimos acessar as urls

def rottenize(title):
    '''
    Esta funcao sustitui os carateres por vazio ou por _
    para depois entrar nos links (urls) dos filmes no rotten
    '''
    title = title.translate(str.maketrans({':': '', "'": '', ".":"", ",":"", 
                                          "-":"_", " ":"_"}))
    
    return title.lower()

In [12]:
# criamos uma lista vazia para armazenar as urls dos filmes do dataset do TMDB
links = []
for tit in df0['original_title'].apply(rottenize):
    links.append('https://www.rottentomatoes.com/m/{}'.format(tit))

In [13]:
# listas para armazenar a informacao
tmdb_id = []

nomes = []
TOMATOMETER = []
TOMATOMETER_rev = []
aud_scr = []
aud_scr_rev = []
main6 = []
main7 = []
main8 = []
main9 = []

genre = []
language = []
director = []
producer = []
writer = []
rel_date_th = []
rel_date_stream = []
box = []
runtime = []
distributor = []
description = []

In [14]:
# name[0], name[2]

In [15]:
# name

In [16]:
# short_info

In [17]:
# j = 1
# driver.get(links[j])

# # se aparecer algun erro no navegador voltamos ao comeco do loop

# # informacoes da descricao do filme
# info_we = driver.find_elements(By.CLASS_NAME, "media-body")
# # informações mais breves da descricao
# short_info = info_we[0].text.split('\n')[2:]

# # informacoes principais do filme: nome, scores e quantidade de reviews
# info_main = driver.find_elements(By.CLASS_NAME, "thumbnail-scoreboard-wrap")
# info_main = info_main[0].text.split('\n')

In [18]:
# info_main

In [19]:
# selecionamos um pedaco da lista de links de filmes comencando pelo "start"
# links = links[start:]

def create_df(n):
    '''
    n: numero de filmes a coletar
    '''  
    # neste dataset iremos armazenar as informacoes extraidas do Rotten Tomatoes
    df = pd.DataFrame()

    for j in range(len(links[:n])):
        
        # mostra quantos filmes ja varreu
        print(j, end="\r", flush=True)
        
        # entramos no site do filme
        driver.get(links[j])
        
        # se aparecer algun erro no navegador voltamos ao comeco do loop
        try:
            # informacoes da descricao do filme
            info_we = driver.find_elements(By.CLASS_NAME, "media-body")
            # informações mais breves da descricao
            short_info = info_we[0].text.split('\n')[2:]

            # informacoes principais do filme: nome, scores e quantidade de reviews
            info_main = driver.find_elements(By.CLASS_NAME, "thumbnail-scoreboard-wrap")
            info_main = info_main[0].text.split('\n')

            # dicionario para armazenar as infos do filme
            short_info_dic = dict()

            # acrescentamos tambem as informacoes principais
            short_info_dic['Nome'] = info_main[0]
            short_info_dic['TOMATOMETER'] = info_main[2]
            short_info_dic['TOMATOMETER reviews'] = info_main[4]
            short_info_dic['AUDIENCE SCORE'] = info_main[5]
            short_info_dic['AUDIENCE SCORE reviews'] = info_main[7]
            
            # colunas auxiliares
            short_info_dic['main6'] = info_main[1]
            short_info_dic['main7'] = info_main[3]
            short_info_dic['main8'] = info_main[6]
            
            try:
                short_info_dic['main9'] = info_main[8]
            except:
                continue
            
        except:
            continue

        # informacoes breves do filme
        for i in range(len(short_info)):
            infos = short_info[i].split(': ')
            try:
                short_info_dic[infos[0]] = infos[1]
            except:
                short_info_dic[infos[0]] = np.nan

        # resumo e descricao do filme
        short_info_dic[info_we[0].text.split('\n')[0]] = info_we[0].text.split('\n')[1]

        # colocamos as informacoes nas listas
        try:
            nomes.append(short_info_dic['Nome'])
            tmdb_id.append(df0.loc[[j],['id']].values[0][0])
        except:
            nomes.append(np.nan)
        
        try:
            TOMATOMETER.append(short_info_dic['TOMATOMETER'])
        except:
            TOMATOMETER.append(np.nan)
        
        try:
            TOMATOMETER_rev.append(short_info_dic['TOMATOMETER reviews'])
        except:
            TOMATOMETER_rev.append(np.nan)
        
        try:
            aud_scr.append(short_info_dic['AUDIENCE SCORE'])
        except:
            aud_scr.append(np.nan)
        
        try:
            main6.append(short_info_dic['main6'])
        except:
            main6.append(np.nan)
            
        try:
            main7.append(short_info_dic['main7'])
        except:
            main7.append(np.nan)
            
        try:
            main8.append(short_info_dic['main8'])
        except:
            main8.append(np.nan)
            
        try:
            main9.append(short_info_dic['main9'])
        except:
            main9.append(np.nan)
        
        try:
            aud_scr_rev.append(short_info_dic['AUDIENCE SCORE reviews'])
        except:
            aud_scr_rev.append(np.nan)
        
        try:
            genre.append(short_info_dic['Genre'])
        except:
            genre.append(np.nan)
        
        try:
            language.append(short_info_dic['Original Language'])
        except:
            language.append(np.nan)
        
        try:
            director.append(short_info_dic['Director'])
        except:
            director.append(np.nan)
        
        try:
            producer.append(short_info_dic['Producer'])
        except:
            producer.append(np.nan)
        
        try:
            writer.append(short_info_dic['Writer'])
        except:
            writer.append(np.nan)
        
        try:
            rel_date_th.append(short_info_dic['Release Date (Theaters)'])
        except:
            rel_date_th.append(np.nan)
        
        try:
            rel_date_stream.append(short_info_dic['Release Date (Streaming)'])
        except:
            rel_date_stream.append(np.nan)
        
        try:
            box.append(short_info_dic['Box Office (Gross USA)'])
        except:
            box.append(np.nan)
        
        try:
            runtime.append(short_info_dic['Runtime'])
        except:
            runtime.append(np.nan)
        
        try:
            distributor.append(short_info_dic['Distributor'])
        except:
            distributor.append(np.nan)
            
        try:
            description.append(short_info_dic['MOVIE INFO'])
        except:
            description.append(np.nan)
            
    # Alimentamos o database

    df = pd.DataFrame() 

    df['tmdb_id'] = tmdb_id
#     df['Movie Name'] = nomes
    df['tomatometer_reviews'] = aud_scr
#     df['AUDIENCE SCORE reviews'] = aud_scr_rev
    # mudou a estrutura do site, entao algumas listas das q foram criadas inicialmente
    # nao coincidem com o nome da coluna
    df['genre_year'] = TOMATOMETER
#     df['TOMATOMETER reviews'] = nomes

    df['movie_name'] = main6
    df['tomatometer_score'] = main7
    df['audience_score'] = main8
    df['audience_reviews'] = main9
    
    df['Genre'] = genre
    df['Original Language'] = language
    df['Director'] = director
    df['Producer'] = producer
    df['Writer'] = writer
    df['Release Date (Theaters)'] = rel_date_th
    df['Release Date (Streaming)'] = rel_date_stream
    df['box office'] = box
    df['Runtime'] = runtime
    df['Distributor'] = distributor
    df['MOVIE INFO'] = description
        
    return df

In [20]:
# # conexao com o database do GCP
# con = psycopg2.connect(host='104.154.213.9' , database='turing_de',
# user = 'postgres', password='pwd')

In [21]:
# df

In [ ]:
# %%timeit
df = create_df(len(links))
# df = create_df(4)

In [ ]:
df

In [ ]:
# df.to_csv("data_output/Rotten_2021_4000.csv", index=False)

In [ ]:
# # @timeit

# df.to_csv("data_output/Rotten_2021_4000.csv", index=False)

In [ ]:
# pd.read_csv("data_output/Rotten_2021_4000.csv").head(50)

In [ ]:
# filtrar por lingua e por ano